# Originial Ochiai
Script that runs the original Ochiai by running the real failing tests from the gzoltar execution. If the bug has no failing tests, it will be skipped. The output is a list with the Ochiai suspiciouness scores per method, one file per bug.

In [7]:
import os
from my_secrets import base_path


paths_dict=  {
    "gzoltar_files_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "gzoltar_files"),
    "output_file": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "ochiaiScores", "originalOchiai"),
    "data_file_path": os.path.join(base_path, "DeepDiveBugReportsWithLogs", "data", "bug_reports_with_stack_traces_details.json"),
    "failing_tests_info_file_name": "failing_tests_info",
    "tests_analysis_results":  os.path.join(base_path,"DeepDiveBugReportsWithLogs",  "data", "rq1_results.json")
}

## Reading the coverage data

In [ ]:
%run ../utils.py

bugs_data = json_file_to_dict(paths_dict["data_file_path"])
tests_analysis_results  = json_file_to_dict(paths_dict["tests_analysis_results"])
bugs = get_list_of_bugs_with_coverage(tests_analysis_results)
coverage_data = {}
for bug in bugs:
    project, bug_id = bug.split("_")
    project_gzoltar_folder = os.path.join(paths_dict["gzoltar_files_path"], project)
    if not os.path.exists(project_gzoltar_folder):
        print("Gzoltar folder not fount for the project "+ project + " Skipping!")
        continue
    if not project in coverage_data.keys():
        coverage_data[project] = {}
    bug_gzoltar_folder = os.path.join(project_gzoltar_folder, bug_id)
    if not os.path.exists(bug_gzoltar_folder):
        print("Gzoltar folder not fount for the bugId "+ bug + " Skipping!")
        print("Skipping!!!!! ")
        continue
    coverage_data[project][bug_id] = {}
    coverage_data[project][bug_id]["statements_covered_per_test"] = read_matrix_file(bug_gzoltar_folder)
    print("Number of tests in bug "+ project+ "-" + bug_id + " - " + str(len(coverage_data[project][bug_id]["statements_covered_per_test"])))
    coverage_data[project][bug_id]["lines_of_code_obj_list"] = read_spectra_file(bug_gzoltar_folder)
    test_names, test_results = read_tests_file(bug_gzoltar_folder)
    coverage_data[project][bug_id]["test_names"] = test_names
    coverage_data[project][bug_id]["test_results"] = test_results
print("Done")

## Running the original Ochiai

In [ ]:
%run ../utils.py
import math

bugs_data = json_file_to_dict(paths_dict["data_file_path"])
failing_tests_info = {}

for project in coverage_data.keys():
    for bug_id in coverage_data[project].keys():
        bug = project + "_"+ bug_id
        print(bug)

        bug_data = bugs_data[project][bug_id]
        coverage = coverage_data[project][bug_id]

        buggy_commit = bug_data["buggy_commit"]
        methods_list = []

        failing_tests = []
        for index, test in enumerate(coverage_data[project][bug_id]["test_names"]):
            if not coverage_data[project][bug_id]["test_results"][index]: # failing test
                failing_tests.append(test)
        failing_tests = list(set(failing_tests))

        if not failing_tests:
            print(f"The bug {project}_{bug_id} does not contain failing tests. Skipping it")
            continue

        print("* Part 1 - Modifying the Gzoltar results to be per method instead of statement")
        coverage["methods_covered_per_test"] = []
        coverage["methods_obj_list"] = []
        method_to_pos = {}

        #Getting the methods list
        for index_t, test_coverage in enumerate(coverage["statements_covered_per_test"]):
            for index_s, statement_instance in enumerate(coverage["statements_covered_per_test"][index_t]):
                lines_of_code_obj_list = coverage["lines_of_code_obj_list"][index_s]
                method_id =  lines_of_code_obj_list["class_name"].replace("$", ".") + "#" + lines_of_code_obj_list["method_name"]
                if method_id not in method_to_pos:
                    method_to_pos[method_id] = len(coverage["methods_obj_list"])
                    coverage["methods_obj_list"].append(method_id)


        # Getting the methods coverage matrix
        for index_t, test_coverage in enumerate(coverage["statements_covered_per_test"]):
            test_name = coverage["test_names"][index_t]
            # Pre=populating the matrix
            coverage["methods_covered_per_test"].append([])
            for method_name in coverage["methods_obj_list"]:
                coverage["methods_covered_per_test"][index_t].append("0")
            # Getting the coverage
            for index_s, statement_instance in enumerate(coverage["statements_covered_per_test"][index_t]):
                lines_of_code_obj_list = coverage["lines_of_code_obj_list"][index_s]
                method_id =  lines_of_code_obj_list["class_name"].replace("$", ".") + "#" + lines_of_code_obj_list["method_name"]
                position = method_to_pos[method_id]
                if coverage["methods_covered_per_test"][index_t][position]!= "1":
                    coverage["methods_covered_per_test"][index_t][position] = statement_instance
        print("Storing the gzoltar results extended per method in a file for future uses")
        store_methods_coverage_in_file(coverage, project, bug_id, paths_dict["gzoltar_files_path"], "test_results_original_ochiai.csv")
        print("store_methods_coverage_in_file operation completed")

        print("* Part 2 - Executing Ochiai")
        methods_ochiai_scores = {}
        for index_m, method_name in enumerate(coverage["methods_obj_list"]):
            n00 = 0
            n01 = 0
            n10 = 0
            n11 = 0
            s_o = 0
            for index_t, test_name in enumerate(coverage["test_names"]):
                if str(coverage["methods_covered_per_test"][index_t][index_m]) == "1":
                    if not coverage["test_results"][index_t]:
                        n11 += 1
                    else:
                        n10 += 1
                else:
                    if not coverage["test_results"][index_t]:
                        n01 += 1
                    else:
                        n00 += 1
            try:
                s_o = n11/math.sqrt((n11+n01)*(n11+n10))
            except ZeroDivisionError:
                s_o = 0
            methods_ochiai_scores[method_name] = s_o

        if project not in failing_tests_info.keys():
            failing_tests_info[project] = {}

        failing_tests_info[project][bug_id] = {}
        failing_tests_info[project][bug_id]["passing_tests_number"] = len(coverage["test_results"]) - len(failing_tests_info)
        failing_tests_info[project][bug_id]["failing_tests_number"] = len(failing_tests)

        print("Number of passing tests: " + str(failing_tests_info[project][bug_id]["passing_tests_number"]))
        print("Number of failing tests: " + str(failing_tests_info[project][bug_id]["failing_tests_number"]) + "\n")
        dict_to_json_file(os.path.join(paths_dict["output_file"], project, bug_id), methods_ochiai_scores)

dict_to_json_file(os.path.join(paths_dict["output_file"],
                               paths_dict["failing_tests_info_file_name"]), failing_tests_info)
print("Execution completed")